# Regression to the Mean: Student Scores and Talent

_Regression to the Mean_ is a statistical feature of data where future observations tend to not be as extreme as previous observations.  That is, we say extreme observations will regress towards the mean in the future.  This phenomenon shows up everywhere from team records to individual performances.

## Setup

In [ ]:
%run ../../utils/notebook_setup.py

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats

import datascience as ds
from datascience import Table
from datascience_stats import linear_fit

## 1. Student Talent
A simple example can be demonstrated through simulation.  Suppose we have 1,000 students take two tests.  The tests are designed to measure the underlying talent of the student.

Student talent is normally distributed centered at 50 and with standard deviation 5.  A histogram of the talent distribution is below.

In [ ]:
np.random.seed(2027)
n = 1000

student_talents = np.random.normal(50, 5, size=n)
plt.hist(student_talents);

### Student Scores
The tests are noisy measurements of the student talent: a talented student will tend to score well but the score can fluctuate.  We model the test performance as Normally distributed with mean 0 and a standard deviation we will vary.

We start with a standard deviation of 2.

In [ ]:
std_score = 2
scores1 = student_talents + np.random.normal(0, std_score, size=n)
scores2 = student_talents + np.random.normal(0, std_score, size=n)
scores = Table().with_columns(['scores1', scores1, 'scores2', scores2])

scores.scatter('scores1', select='scores2')

### A Regression Prediction for Test 2 Scores

Since there isn't too much noise in the tests, the tests do a decent job of measuring the performance and the test scores are obviously strongly correlated through the underlying student talent.

Suppose we had a new set of students and administered the first test.  If we want a prediction for a score on the second test, we can use the above data to build a model through linear regression.

\begin{align*}
    \text{Test 2 Score} & = \alpha + \beta \times \text{Test 1 Score} \\
        & = \alpha + \beta \times (\text{Test 1 Score} - \text{Test 1 Avg} + \text{Test 1 Avg}) \\
        & =  (\alpha + \beta \times \text{Test 1 Avg}) + \beta \times (\text{Test 1 Score} - \text{Test 1 Avg}) \\
    & = \tilde{\alpha} + \beta \times (\text{Test 1 Score} - \text{Test 1 Avg})
\end{align*}

In [ ]:
params, _, _ = linear_fit(scores1, scores2, constant=True)
intercept, slope = params

alpha = intercept
alpha_tilde = intercept + slope * np.mean(scores1)
beta = slope

print(f"Avg Score 1: {np.mean(scores1):.3f}")
print(f"Avg Score 2: {np.mean(scores2):.3f}")
print()
print(f"alpha: {alpha:.3f}  alpha_tilde: {alpha_tilde:.3f}  beta: {beta:.3f}")
print(f"Predicted Score 2 = {beta:.3f} x (Score 1) + {alpha:.3f}")
print(f"Predicted Score 2 = {beta:.3f} x (Score 1 - Avg Score) + {alpha_tilde:.3f}")

According to the regression model, the predicted score should not be score on Test 1.  Instead, the predicted score for Test 2 should be closer to the mean (about 50) than the observed score on Test 1.  

### Larger Standard Deviation of Scores

What happens if the test scores are noisier?  We can increase the standard deviation of the scores to see what happens.

If we compare this model to the previous model that had less noise, we see that the model values the score from Test 1 less (smaller slope coefficient) and therefore predictions are even closer to the mean.

In [ ]:
std_score = 5
scores1 = student_talents + np.random.normal(0, std_score, size=n)
scores2 = student_talents + np.random.normal(0, std_score, size=n)
scores = Table().with_columns(['scores1', scores1, 'scores2', scores2])

scores.scatter('scores1', select='scores2')

params, _, _ = linear_fit(scores1, scores2, constant=True)
intercept, slope = params

print(f"Avg Score 1: {np.mean(scores1):.3f}")
print(f"Avg Score 2: {np.mean(scores2):.3f}")
print(f"Predicted Score 2 = {slope:.3f} x (Score 1) + {intercept:.3f}")
print(f"Predicted Score 2 = {slope:.3f} x (Score 1 - Avg Score) + {intercept + slope * np.mean(scores1):.3f}")

And if we go extreme and have a ridiculously large standard deviation of scores, the model basically does not care what the score is on Test 1 and will mainly just predict the mean.

In [ ]:
std_score = 20
scores1 = student_talents + np.random.normal(0, std_score, size=n)
scores2 = student_talents + np.random.normal(0, std_score, size=n)
scores = Table().with_columns(['scores1', scores1, 'scores2', scores2])

scores.scatter('scores1', select='scores2')

params, _, _ = linear_fit(scores1, scores2, constant=True)
intercept, slope = params

print(f"Avg Score 1: {np.mean(scores1):.3f}")
print(f"Avg Score 2: {np.mean(scores2):.3f}")
print(f"Predicted Score 2 = {slope:.3f} x (Score 1) + {intercept:.3f}")
print(f"Predicted Score 2 = {slope:.3f} x (Score 1 - Avg Score) + {intercept + slope * np.mean(scores1):.3f}")

### Small Score Standard Deviation

If we go the other way, we see that the score on Test 1 is highly predictive of the score on Test 2 and the model will more or less just use the Test 1 score as the prediction.

In [ ]:
std_score = .01
scores1 = student_talents + np.random.normal(0, std_score, size=n)
scores2 = student_talents + np.random.normal(0, std_score, size=n)
scores = Table().with_columns(['scores1', scores1, 'scores2', scores2])

scores.scatter('scores1', select='scores2')

params, _, _ = linear_fit(scores1, scores2, constant=True)
intercept, slope = params

print(f"Avg Score 1: {np.mean(scores1):.3f}")
print(f"Avg Score 2: {np.mean(scores2):.3f}")
print(f"Predicted Score 2 = {slope:.3f} x (Score 1) + {intercept:.3f}")
print(f"Predicted Score 2 = {slope:.3f} x (Score 1 - Avg Score) + {intercept + slope * np.mean(scores1):.3f}")

### Regression to the Mean

The above models demonstrate the concept of _Regression to the Mean_.  

Our regression models were given as:
$$
    \text{Test 2 Score} = \hat\alpha + \hat\beta \cdot \text{Test 1 Score}
$$

The formulas for the coefficients are:
$$
    \hat\alpha = \text{Test 2 Avg} - \hat\beta\cdot \text{Test 1 Avg}
$$
and,
$$
    \hat\beta = \mathrm{Corr}(\text{Test 1 Score}, \text{Test 2 Score})\cdot \frac{\text{Std Test 2 Score}}{\text{Std Test 1 Score}}
$$

Plugging in, we get
$$
    \text{Test 2 Score} = \text{Test 2 Avg} + \hat\beta\cdot\left(\text{Test 1 Score} - \text{Test 1 Avg} \right)
$$

If our two test scores feature a similar standard deviation, then the driver of the slope $\hat\beta$ is the correlation between the two scores.  And the driver of the correlation of the test scores is the overall magnitude of the standard deviations of the test scores.

The term Regression to the Mean comes from the fact that our predicted values will be closer to the mean than the observed values due to the fact that $\hat\beta$ is less than 1.  Multiplication by $\hat\beta$ is how we regress observations towards the mean.

In general, regression to the mean will entail building an estimate of the form
$$
    \text{Regressed Estimate} = w \cdot \text{Observation} + (1 - w) \cdot \text{Mean}
$$
where $w$ is between 0 and 1.  For $w$ close to 1, we do not regress much because we think the observation is more important.  For $w$ close to 0, we regress a lot because we think the noise in the observation renders it not informative and the mean is a better guess.

The next cell should help convince you that the order of the regression (predicting Test 2 from Test 1) doesn't matter.  In general, predictions should not be as far from the mean as the observations.

For more on this, see Sec 15.2 of Inferential Thinking: https://www.inferentialthinking.com/chapters/15/2/regression-line.html

In [ ]:
std_score = 5
scores1 = student_talents + np.random.normal(0, std_score, size=n)
scores2 = student_talents + np.random.normal(0, std_score, size=n)
scores = Table().with_columns(['scores1', scores1, 'scores2', scores2])

scores.scatter('scores1', select='scores2')

# Flip the variables and regress scores1 onto scores2
params, _, _ = linear_fit(scores2, scores1, constant=True)
intercept, slope = params

print(f"Avg Score 1: {np.mean(scores1):.3f}")
print(f"Avg Score 2: {np.mean(scores2):.3f}")
print(f"Predicted Score 1 = {slope:.3f} x (Score 2) + {intercept:.3f}")
print(f"Predicted Score 1 = {slope:.3f} x (Score 2 - Avg Score) + {intercept + slope * np.mean(scores2):.3f}")

## 2. Updating Beliefs

At first, this section might appear to be unrelated.  But we will see how we can relate the concept of prior belief, updates, and Bayesian thinking with regression to the mean.

We're going to do some coin tossing.  Our coin will have a probability $p$ of coming up heads.  We don't know $p$ and we're going to try to come up with an estimate for $p$.

`CoinFlipper` is a class that will help us flip the coins and keep track of the results and make some handy plots.

In [ ]:
from datascience_topic import CoinFlipper

### Class Opinion on $p$

The coin has some random probability of coming up heads that we don't know so we might as well start tossing the coin to get a read on what $p$ could be.  If it's large, we should see a lot of heads.  If it's small, a lot of tails.  There's only one way to find out though.

Before we do this, we can take a poll for what people _think_ the probability is.  As we toss the coin, we'll keep asking for guesses.

In [ ]:
# create a flipper object
flipper = CoinFlipper()

In [ ]:
n = 10
flipper.toss_coin(n=n)
flipper.report()
flipper.plot_tosses(figsize=(6,6))

So what was $p$?  And how many heads did we expect to see?

In [ ]:
print(flipper.p, flipper.p * flipper.num_tosses)

### Evolving Distribution of Belief

Since I didn't say anything about $p$ at the start, our initial belief about $p$ was flat or uninformed.  We model this through two simple parameters: $\text{Expected Heads}$ and $\text{Expected Tails}$, or just $H$ and $T$ for short.  Our flat, indifferent, agnostic, or uninformed belief is given by $H=1$ and $T=1$.  Or _best guess_ is the mean of the belief distribution which is given by the formula
$$
    \text{Best Guess for $p$} = \frac{H}{H + T}
$$
For $H=T=1$, our best guess is $p = \frac12$.

We call it the best guess because it will minimize expected error based on whatever $p$ may be.

In [ ]:
flipper = CoinFlipper(expected_heads=1, expected_tails=1)
flipper.report(report_belief=True)
flipper.plots()

### Tossing Coins and Updating the Belief Distribution

As we toss the coin, we use the observations of heads and tails to update our beliefs.  The updating formula is very simple:
$$
    H_{new} = H_{old} + \text{Number of Heads tossed},\quad T_{new} = T_{old} + \text{Number of Tails tossed}
$$

Our updated best guess is:
\begin{align*}
    \text{Updated Best Guess for $p$} 
        & = \frac{H_{new}}{H_{new} + T_{new}} \\
        & = \frac{H_{old} + \text{Number of Heads tossed}}
        {H_{old} + \text{Number of Heads tossed} + T_{old} + \text{Number of Tails tossed}}\\
        & = \frac{H_{old} + \text{Number of Heads tossed}}
        {H_{old} + T_{old} + \text{Number of tosses}}
\end{align*}


In [ ]:
n = 100000
flipper.toss_coin(n=n)
flipper.report(report_belief=True)
flipper.plots()

### Stronger Prior Beliefs

We do not have to start with a flat, uninformed prior.  We may know something about our coin!

Below, we start with $H=73$ and $T=210$.  We take $p=.370$ so that we can see how things behave more specifically.

So what does it mean that $H=73$ and $T=210$?  Well, for one, our initial best guess for $p$ (if we didn't know it)  without any coin tosses is 
$$
    \text{Initial Best Guess} = \frac{73}{283} = 0.258
$$
So $H$ and $T$ encode where the center of the distribution is (.258).  It also encodes the shape, ie. how tight around the center the distribution is.  Large values of $H$ and $T$ mean we have stronger beliefs and therefore a tighter distribution.  

In [ ]:
flipper = CoinFlipper(expected_heads=75, expected_tails=215, p=.370)
flipper.report(report_belief=True)
flipper.plots()

As we make observations, our best guess will have the formula,
$$
    \text{Best Guess after $N$ tosses} = \frac{73 + \text{Number of Heads}}{283 + N}
$$

In [ ]:
n = 600
flipper.toss_coin(n=n)
flipper.report(report_belief=True)
flipper.plots()

### Best Guess and Regression to the Mean

Let's look closer at the formula for the best guess:
$$
    \text{Best Guess after $N$ tosses} = \frac{H_{initial} + \text{Number of Heads}}{H_{initial} + T_{initial} + N}
$$

With a little algebra, we get:
$$
    \text{Best Guess after $N$ tosses} 
        = \frac{N}{H_{initial} + T_{initial} + N}\cdot\frac{\text{Number of Heads}}{N} 
        + \frac{H_{initial} + T_{initial}}{H_{initial} + T_{initial} + N}\cdot\frac{H_{initial}}{H_{initial} + T_{initial}}
$$
Or...
\begin{gather*}
    \text{Best Guess after $N$ tosses}
        = w_N \cdot \text{Proportion of Heads observed} + (1 - w_N) \cdot \text{Initial Best Guess}\\
    w_N = \frac{N}{H_{initial} + T_{initial} + N}
\end{gather*}
Note that $w_N$ is between 0 and 1.

This is precisely a formula for regression to the mean!  The mean is our initial best guess and we "regress" our observed frequency towards the mean.  As we observe more and more data though, we regress less and less because $w_N$ gets closer and closer to 1 as $N$ grows.

In [ ]:
w_N = 10000 / (75 + 215 + 10000)
w_N

### Beta Belief Distribution

So far, we haven't said anything about what this belief distribution is.  It's actually known as a Beta distribution and the function plotted above has the form,
$$
    K \cdot x^{H - 1} \cdot (1 - x)^{T - 1}
$$
where $K$ is just a constant that ensures the integral of the function from 0 to 1 equals 1.  

The parameters $H$ and $T$ (also often labeled as $a$ and $b$ instead) clearly govern the location of the center of the distribution as well as the shape.  And as they increase, create a tighter distribution.

In [ ]:
x = np.linspace(0, 1, 1000)
H = 1000
T = 200
plt.plot(x, x**(H - 1) * (1 - x)**(T - 1));


## 3. Regression to the Mean, Baseball, and Batting Average

Let's apply the above to baseball and projecting a player's batting average after only a few at-bats.

We load the Lahman tables as usual.  The hardest part is we need to build an initial guess for batting average that will serve as the mean we will regress towards.  Here is how we will do that:

1. Collect historical data on players and their career batting averages.
2. Filter out pitchers and other batters who did not bat much
3. Estimate $H$ and $T$ from the historical data.
4. Use the estimated $H$ and $T$ to form a regressed estimate of batting average.  This is an _Empirical Bayes_ estimation of the estimated batting average.

### Loading Lahman Data

In [ ]:
lahman_bat = Table.read_table('lahman_batting.csv') 
lahman_bat['PA'] = lahman_bat['AB'] + lahman_bat['BB'] + lahman_bat['HBP'] + \
    lahman_bat['SF'] + lahman_bat['SH']
lahman_pitch = Table.read_table('lahman_pitching.csv') 

In [ ]:
lahman_bat.show(5)

### Compute Career Totals

For batters, we compute total hits and at-bats.  For pitchers, we compute total batters faced (`BFP`).  

In [ ]:
batting_totals = lahman_bat.select('playerID', 'H', 'AB').\
    group('playerID', collect=sum).\
    relabel('H sum', 'H').\
    relabel('AB sum', 'AB')
pitching_totals = lahman_pitch.select('playerID', 'BFP').\
    group('playerID', collect=sum).\
    relabel('BFP sum', 'Batters Faced')

We need to know who the pitchers are because they can grossly distort how we view the distribution of batting averages.  The histogram shows just how messy the raw batting averages are.

In [ ]:
batting_totals['BA'] = batting_totals['H'] / (batting_totals['AB'] + 1)  # add 1 AB so we don't divide by 0
batting_totals.hist('BA', bins=50)

### Merge Pitchers and Batters

In [ ]:
from datascience_utils import merge

merged_players = merge(batting_totals, pitching_totals, 'playerID', how='outer', fillna=True)
merged_players.show(20)

### Drop Pitchers
We figure out who the batters are by taking anyone with more at-bats and batters faced.

In [ ]:
more_AB_than_BFP = merged_players['AB'] > merged_players['Batters Faced']
batters = merged_players.where(more_AB_than_BFP)

### Take Batters with at least 500 AB

We want a more stable notion of batting average so we need to restrict to batters with a minimum number of AB.  500 is probably good enough since that represents about 1 season.

In [ ]:
batters_500AB = batters.where('AB', ds.are.above_or_equal_to(500))
batters_500AB.hist('BA', bins=50)

### Build the Belief Distribution

The function `fit_beta_belief` can take in observed frequencies/rates and fit the Beta belief distribution.  The function `plot_beta_belief` can plot the belief distribution.

In [ ]:
from datascience_topic import fit_beta_belief, plot_beta_belief, plot_beta_belief_and_batting_avg

H, T = fit_beta_belief(batters_500AB['BA'])
plot_beta_belief(H, T)
H, T

How does our fitted distribution compare to the historical data on batting average?  The helper function `plot_beta_belief_and_batting_avg` can take care of that.

It looks like a pretty great fit!

In [ ]:
plot_beta_belief_and_batting_avg(H, T, batters_500AB)

### Returning to the CoinFlipper

We can use the coin flipper to explore how our belief updates and how we regress to the mean after a few at-bats.

In [ ]:
flipper = CoinFlipper(expected_heads=H, expected_tails=T, p=.370)
flipper.report(report_belief=True)
flipper.plots()

In [ ]:
flipper.toss_coin(n=20)
flipper.report(report_belief=True)
flipper.plots()

## Projections: Marcel the Monkey

How can we use regression to the mean for the purposes of projecting statistics in an upcoming season?  Tom Tango's very simple projection system, Marcel, is a great way to explore how to use recency weighting and regression to the mean to build a projection.

And despite the relatively lack of complexity, it apparently performs pretty well!

### Projecting Mike Trout's HRs

The best way to learn the Marcel system is to walk through the computation.  Let's pull up Mike Trout's data and forecast his HRs for 2018.

In [ ]:
trout = lahman_bat.where('playerID', 'troutmi01')
trout

### HRs and PAs

In order to forecase Mike Trout's HRs, we need his PAs and HRs for the last three seasons.  For each of those seasons, we compute his HR rate as $\mathit{HR} / \mathit{PA}$.

In [ ]:
trout_HRs_last3 = trout['HR'][-3:]
trout_PAs_last3 = trout['PA'][-3:]
trout_HRrate_last3 = trout_HRs_last3 / trout_PAs_last3
trout_HRs_last3, trout_PAs_last3, trout_HRrate_last3

### Recency Weighting

Marcel uses recency weighting to base the projection on performance from the player.  The weights for the previous three seasons, from oldest to most recent, are given below.

These weightings are the modeller's discretion and Tango does not offer too much insight other than it seems to work well.

In [ ]:
marcel_weights = np.array([3, 4, 5]) / 12
marcel_weights

### Trout's Weighted Average HR Rate and PA

We want to compute Mike Trout's average HR rate and average PA over the last three years.  But we use the Marcel weights to create the recency weighting.  We also need to weight the relevance by how many PAs Mike Trout had in each season.  Last season, he had many fewer PAs so even though it's the most recent, it needs to be downgraded in importance due to it being a relatively smaller sample.

$$
    \text{Weighted Avg PA} = \sum_{year} \text{PA}_{year} \cdot \text{Marcel Weight}_{year}
$$

$$
    \text{Weighted Avg HR Rate} = \sum_{year} \frac{\text{PA}_{year}}{\text{Weighted Avg PA}} \cdot \text{HR Rate}_{year} \cdot \text{Marcel Weight}_{year}
$$

In [ ]:
trout_weighted_avg_PA = np.sum(trout_PAs_last3 * marcel_weights)
trout_weighted_avg_PA

In [ ]:
pa_year_weight = trout_PAs_last3 / trout_weighted_avg_PA
pa_year_weight

In [ ]:
trout_weighted_HRrate = np.sum(trout_HRrate_last3 * marcel_weights * pa_year_weight)
trout_weighted_HRrate

### League Averages

The league average HR rate was computed offline separately.  The values are given below.

We need to compare Trout to a league average player given the same opportunity as Trout had.  We thus use the same PA weightings but swap out Trout's HR rates for the league average rates.

In [ ]:
lg_avg_HRrate = np.array([ 0.02740387,  0.0311592 ,  0.03376997])

In [ ]:
lg_avg_weighted_HRrate = np.sum(lg_avg_HRrate * marcel_weights * pa_year_weight)
lg_avg_weighted_HRrate

### Regression to the Mean Formula

The regresion to the mean formula is simple and one we recognize from above:
$$
    \text{Marcel Projection} = w \cdot \text{Trout Weighted HR Rate} +
        (1 - w) \cdot \text{League Avg Weighted HR Rate}
$$

Tango calls the quantity $w$ _reliability_.  The formula for reliability is simple:
$$
    w = \frac{\text{Trout Weighted Avg PA}}{\text{Trout Weighted Avg PA} + 100}
$$
If we think about this from our Beta Belief distribution, Tango's reliability is based on a prior belief that $H + T = 100$.

It turns out that this is same formula for any statistic in the Marcel system.  A natural extension would be the hone the quantities $a$ and $b$ to achieve better regression to the mean performance.

In [ ]:
w = trout_weighted_avg_PA / (trout_weighted_avg_PA + 100)
w

#### Trout's HR Rate Projection

In [ ]:
trout_HRrate_projection = w * trout_weighted_HRrate + \
    (1 - w) * lg_avg_weighted_HRrate
trout_HRrate_projection

### Projecting PA

To get a HR projection, we need a PA projection.  Tango offers up a simple formula:
$$
    \text{Projected PA} = .5 \cdot \text{Previous Year's PA} + .1 \cdot \text{Second Previous Year's PA} + 200
$$

In [ ]:
PA_proj_weights = np.array([0., .1, .5])
trout_PA_projection = np.sum(trout_PAs_last3 * PA_proj_weights) + 200
trout_PA_projection

### Trout's Raw HR Total Projection

In [ ]:
trout_HR_projection = trout_HRrate_projection * trout_PA_projection
trout_HR_projection

### Age Adjustment

Tango includes a rudimentary age adjustment:
$$
    \text{Age Adjustment} = \begin{cases} 
        0.003 * (29 - \text{Age}) & \text{Age} > 29\\
        0.006 * (29 - \text{Age}) & \text{Age} <= 29
    \end{cases}    
$$

The final projection is given by:
$$
    \text{HR Projection} = \text{Projected PA} \cdot \text{Projected HR Rate} \cdot (1 + \text{Age Adjustment})
$$

In [ ]:
trout_age = 27
age_adjust = 0.003 * (29 - trout_age) if trout_age > 29 \
    else 0.006 * (29 - trout_age)
trout_HR_aged_projection = trout_HR_projection * (1 + age_adjust)
trout_HR_aged_projection